# LakeFS

LakeFS est un outil de contrôle de version pour les données. Il offre des fonctionnalités pareilles à celles de Git (CI/CD, repositories, commits ...). Il permet de contrôler de données hébergées sur GCS, AWS S3, Azure Blob Storage et n'importe quel service de stockage de données qui implémente une interface S3.

**Installer la CLI de lakefs ``lakectl``**

In [3]:
!sudo sh install-lakectl.sh
!lakectl --version

**Connecter sur lakefs**

Il faut d'abord lancer le service LakeFS sur DataLab et désactiver le paramètre ``security.allowlist.enabled``.
<br> La sortie de Jupyter n'est pas interactive donc il faut exécuter la commande ``lakectl config`` dans une terminale. 

In [ ]:
!lakectl config

**Création d'un repository**

Les repositories de LakeFS sont identifiés par un url qui sous la forme: ``lakefs://repository-name``.
<br> Le deuxième champ indique le chemin dans lequel LakeFS va enregister les données.
<br> LakeFS crée par défaut une branche ``main``.

In [95]:
#lakectl repo create <repository uri> <storage namespace> [flags]
!lakectl repo create lakefs://tutorial s3://mbenxsalha/lakefs

Repository: lakefs://tutorial-2
Repository 'tutorial-2' created:
storage namespace: s3://mbenxsalha/lakefs_2
default branch: main
timestamp: 1667208284


In [96]:
!lakectl repo list

+------------+-------------------------------+------------------+--------------------------+
| REPOSITORY | CREATION DATE                 | DEFAULT REF NAME | STORAGE NAMESPACE        |
+------------+-------------------------------+------------------+--------------------------+
| tutorial   | 2022-10-31 07:14:35 +0000 UTC | main             | s3://mbenxsalha/lakefs   |
| tutorial-2 | 2022-10-31 09:24:44 +0000 UTC | main             | s3://mbenxsalha/lakefs_2 |
+------------+-------------------------------+------------------+--------------------------+



**Python Client**

LakeFS offre aussi un client Python qui permet de manipuler les repositories.

In [ ]:
!pip install lakefs_client

In [97]:
import lakefs_client
from lakefs_client import models
from lakefs_client.client import LakeFSClient
# lakeFS credentials and endpoint
configuration = lakefs_client.Configuration()
configuration.username = 'AKIAJGL4O/EXAMPLE'
configuration.password = 'LXrz09Aaf+i0U5fNoo/SECRET-ACESS-KEY'
configuration.host = 'https://user-mbenxsalha-930372.user.lab.sspcloud.fr'

client = LakeFSClient(configuration)
client.repositories.get_repository("tutorial")

{'creation_date': 1667208284,
 'default_branch': 'main',
 'id': 'tutorial-2',
 'storage_namespace': 's3://mbenxsalha/lakefs_2'}

Dans ce tutoriel, on va utiliser un tableau csv qui existe sous le chemin ``s3/mbenxsalha/diffusion/DiamondsPrices.csv``. 
<br> On va l'importer dans notre repository.

In [98]:
#import data to repository
import os
import boto3

s3 = boto3.client('s3',endpoint_url='https://minio.lab.sspcloud.fr/')
data = s3.get_object(Bucket="mbenxsalha", Key="diffusion/DiamondsPrices.csv")["Body"]
client.objects.upload_object(repository='tutorial-2', branch='main', path="data.csv", content=data)



{'checksum': 'eb12d9e0556c9d957c275852dc9acc2d',
 'content_type': 'application/octet-stream',
 'mtime': 1667208306,
 'path': 'data.csv',
 'path_type': 'object',
 'physical_address': 's3://mbenxsalha/lakefs_2/b47793de50f046ffb0993b6eabf5dfb5',
 'size_bytes': 2448482}

**boto3**

Il est possible aussi de lire les données d'un repository en utilisant la librairie boto3. Pour ce faire, il faut configurer un client s3. Le paramètre ``endpoint_url`` fait référence au lien du service LakeFS et les paramètres ``aws_access_key_id`` et ``aws_secret_access_key`` correspondent aux paramètres de LakeFS (ceux utilisés dans lakectl config).

Un objet dans un repository ``repo`` et une branche ``branch`` est accessible via les paramètres ``Bucket = repo`` et  ``Key = branch/object_relative_path_in_repo``

In [99]:
#read data from repository
import pandas as pd

s3 = boto3.client('s3',
    endpoint_url='https://user-mbenxsalha-930372.user.lab.sspcloud.fr/',
    aws_access_key_id='AKIAJGL4O/EXAMPLE',
    aws_secret_access_key='LXrz09Aaf+i0U5fNoo/SECRET-ACESS-KEY')

data = s3.get_object(Bucket="tutorial", Key="main/data.csv")["Body"]

df = pd.read_csv(data)
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


**Commit**

In [100]:
#lakectl commit <branch uri> [flags]
!lakectl commit lakefs://tutorial/main --message "add raw data"

Branch: lakefs://tutorial-2/main
Commit for branch "main" completed.

ID: e49fa6a607bb187b93eda6b7956dccb28c5e948074ec8c572c03f6074155e2f5
Message: add raw data
Timestamp: 2022-10-31 09:25:29 +0000 UTC
Parents: 721be364f770968707792189af16e3e8e57714f6b1e9e895655452103db274de



**Créer une nouvelle branche**

In [102]:
#lakectl branch create <branch uri> -s <source ref uri> [flags]
!lakectl branch create lakefs://tutorial-2/transformed-data -s lakefs://tutorial-2/main

Source ref: lakefs://tutorial-2/main
created branch 'transformed-data' e49fa6a607bb187b93eda6b7956dccb28c5e948074ec8c572c03f6074155e2f5


On supprime les lignes contenant des valeurs manquantes et on crée une nouvelle colonne ``symmetry``.

In [ ]:
df = df.dropna(axis = 0)

In [106]:
import io
with io.StringIO() as csv_buffer:
    df_trans.to_csv(csv_buffer, index=False)
    s3.put_object(Bucket="tutorial", Key="transformed-data/data.csv", Body=csv_buffer.getvalue())

In [108]:
!lakectl commit lakefs://tutorial-2/transformed-data --message "clean data"

Branch: lakefs://tutorial-2/transformed-data
Commit for branch "transformed-data" completed.

ID: a534d8dd3b80ea7f2b18278726b7fcfd41df5eda5cbee33eb1290df7da37c860
Message: clean data
Timestamp: 2022-10-31 09:26:51 +0000 UTC
Parents: e49fa6a607bb187b93eda6b7956dccb28c5e948074ec8c572c03f6074155e2f5



In [104]:
df.corr()

/tmp/ipykernel_126/1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,carat,depth,table,price,x,y,z
carat,1.000000,0.028224,0.181618,0.921591,0.975094,0.951722,0.953387
depth,0.028224,1.000000,-0.295779,-0.010647,-0.025289,-0.029341,0.094924
table,0.181618,-0.295779,1.000000,0.127134,0.195344,0.183760,0.150929
price,0.921591,-0.010647,0.127134,1.000000,0.884435,0.865421,0.861249
x,0.975094,-0.025289,0.195344,0.884435,1.000000,0.974701,0.970772
y,0.951722,-0.029341,0.183760,0.865421,0.974701,1.000000,0.952006
z,0.953387,0.094924,0.150929,0.861249,0.970772,0.952006,1.000000


On modifie le tableau en créant des nouveaux colonnes.

In [ ]:
df_trans = pd.get_dummies(df)

df_trans['symmetry'] = df['x'] / df['y']
df_trans.head(2)

**Sauvegarder les modifications**

In [115]:
import io
with io.StringIO() as csv_buffer:
    df_trans.to_csv(csv_buffer, index=False)

    response = s3.put_object(
        Bucket="tutorial", Key="transformed-data/data.csv", Body=csv_buffer.getvalue()
    )

In [116]:
!lakectl commit lakefs://tutorial/transformed-data --message "transform data"

Branch: lakefs://tutorial-2/transformed-data
Commit for branch "transformed-data" completed.

ID: 433ee1bae2acfd2ffe76bdc8f3e1379d762e7f9fe65a088a062af686fb337bf3
Message: transform data
Timestamp: 2022-10-31 09:29:08 +0000 UTC
Parents: a534d8dd3b80ea7f2b18278726b7fcfd41df5eda5cbee33eb1290df7da37c860



**Revert**

Une fonctionnalité importante de LakeFS est d'annuler un commit et de revenir en arrière. Le deuxième champ indique l'Id du commit à annuler. Vous pouvez le trouver sur l'UI de LakeFS.

In [118]:
#lakectl branch revert <branch uri> <commit ref to revert> [<more commits>...] [flags]
!lakectl branch revert -y lakefs://tutorial/transformed-data 96b70673a909666b250a8cfe71e604baf2b9d6ba3cf6acc8c5a5cdb6dbabc646

Branch: lakefs://tutorial-2/transformed-data
commit 96b70673a909666b250a8cfe71e604baf2b9d6ba3cf6acc8c5a5cdb6dbabc646 successfully reverted
